# NFL Data Scraping Project
This is a notebook that will describe the web scraping process for NFL team statistics

To start we need to figure out what information and where to find that information. For this project I want to analyze how regular season statistics for a team impact their performance during the regular season and in the playoffs.
- To find the regular season statistics I will be using the information provided by the NFL here: **https://www.nfl.com/stats/team-stats/**
- To find the win and loss information for teams I used information from this link: **https://www.teamrankings.com/nfl/trends/win_trends/**

Let's start by gathering the win loss data

### Packages
For this project I will be using selenium and using beautiful soup to conduct web scraping.

In [ ]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from urllib.parse import urljoin
import requests as r
import pandas as pd

url = "https://www.teamrankings.com/nfl/trends/win_trends/?sc=is_regular_season"
response = r.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
specific_div = soup.find('table') # finding the table with the record of each team
rows = specific_div.find_all('tr') # get the rows of data

# Now that we have the data as a list of rows, we can parse the data to construct a data frame
data = []
for row in rows:
        cells = row.find_all(['td', 'th'])  # 'td' for regular cells, 'th' for header cells
        row_data = [cell.text.strip() for cell in cells] #extract the contents in each cell
        data.append(row_data)
columns = data[0]
df = pd.DataFrame(data[1:], columns=columns)
print(df)


We have now parsed the data for 1 year. The next challenge is to change the filters to get postseason data, and then to change the year filter to get data for each year. When inspecting the html code we can see that there a 'div class=filter' which holds the different filter that we can change. When a year filter is changed the url changes. For examples *https://www.teamrankings.com/nfl/trends/win_trends/?sc=is_regular_season&range=yearly_2022&range=yearly_2022*. Using the final parameter "&range=yearly_2022" we can adjust the url to select the year to scrape the data


In [35]:
url = url + v
year = 2022
url = url + str(year)
print(url)

https://www.teamrankings.com/nfl/trends/win_trends/?sc=is_regular_season&range=yearly_2022&range=yearly_2022


Now the goal is to create a function of the above code, and iterate through each year to pull data. Note that for the data to be scraped, we want a full year of data. As of this time the 2023 season is going on, so we will look to pull data up to 2022.

In [ ]:
def read_data(year):
    # Reads the data from the website extracting the values of the table based on the year
    # Param: year - year of data to be collected

    url = "https://www.teamrankings.com/nfl/trends/win_trends/?sc=is_regular_season&range=yearly_"
    url = url + year
    response = r.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    specific_div = soup.find('table') # finding the table with the record of each team
    rows = specific_div.find_all('tr') # get the rows of data

    # Now that we have the data as a list of rows, we can parse the data to construct a data frame
    data = []
    for row in rows:
            cells = row.find_all(['td', 'th'])  # 'td' for regular cells, 'th' for header cells
            row_data = [cell.text.strip() for cell in cells] #extract the contents in each cell
            data.append(row_data)
    columns = data[0]
    df = pd.DataFrame(data[1:], columns=columns)
read_data("2003")

Now that we have made a function, we want to have 1 table with data from all possible years. To do this we need to identify each data point by year. Therefore we need to adjust our original function to add column for year.

In [67]:
def read_data(year):
    # Reads the data from the website extracting the values of the table based on the year
    # Param: year - year of data to be collected

    url = "https://www.teamrankings.com/nfl/trends/win_trends/?sc=is_regular_season&range=yearly_"
    url = url + str(year)
    response = r.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    specific_div = soup.find('table') # finding the table with the record of each team
    rows = specific_div.find_all('tr') # get the rows of data

    # Now that we have the data as a list of rows, we can parse the data to construct a data frame
    data = []
    for row in rows:
            cells = row.find_all(['td', 'th'])  # 'td' for regular cells, 'th' for header cells
            row_data = [cell.text.strip() for cell in cells] #extract the contents in each cell
            data.append(row_data)
    columns = data[0]
    df = pd.DataFrame(data[1:], columns=columns)
    df["Year"] = year #helps identify the data points based on year
    return df
read_data("2004")

,Team,Win-Loss Record,Win %,MOV,ATS +/-,Year
0,Pittsburgh,15-1-0,93.8%,7.6,+4.8,2004
1,New England,14-2-0,87.5%,11.1,+4.2,2004
2,Philadelphia,13-3-0,81.3%,7.9,+1.8,2004
3,Indianapolis,12-4-0,75.0%,10.7,+5.4,2004
4,LA Chargers,12-4-0,75.0%,8.3,+9.0,2004
5,Atlanta,11-5-0,68.8%,0.2,-2.0,2004
6,NY Jets,10-6-0,62.5%,4.5,+2.3,2004
7,Green Bay,10-6-0,62.5%,2.8,+0.8,2004
8,Denver,10-6-0,62.5%,4.8,+0.1,2004
9,Seattle,9-7-0,56.3%,-0.1,-3.8,2004


Now that we can identify the year for each data table, we need to loop through each year and pull the data for the regular season. To get the data for years, the __datetime__ package will be used to always get the current year. We will loop through years from 2003 to the year before the current year. All of this data will then be placed into 1 table

In [69]:
import datetime as dt
all_data = []
current_year = int(dt.datetime.today().strftime("%Y"))
for i in range(2003, current_year):
    x = read_data(i)
    all_data.append(x) # adding data frame objects to a list

regular_seasonn_data = pd.concat(all_data) # using pandas concat function which is more efficient than for loop
regular_seasonn_data


,Team,Win-Loss Record,Win %,MOV,ATS +/-,Year
0,New England,14-2-0,87.5%,6.9,+5.1,2003
1,Kansas City,13-3-0,81.3%,9.5,+3.7,2003
2,Philadelphia,12-4-0,75.0%,5.4,+3.1,2003
3,Tennessee,12-4-0,75.0%,6.9,+3.0,2003
4,Indianapolis,12-4-0,75.0%,6.9,+3.4,2003
...,...,...,...,...,...,...
27,LA Rams,5-12-0,29.4%,-4.5,-3.0,2022
28,Indianapolis,4-12-1,25.0%,-8.1,-6.6,2022
29,Arizona,4-13-0,23.5%,-6.4,-2.5,2022
30,Houston,3-13-1,18.8%,-7.7,-0.4,2022


Now that we have the regular season win loss information, we can adjust our function to get information about the playoff games. We can do this by adding a parameter for the type of game we want to search for. 

In [ ]:
def read_data(year, game_type):
    # Reads the data from the website extracting the values of the table based on the year
    # Param: year - year of data to be collected

    url = f"https://www.teamrankings.com/nfl/trends/win_trends/?sc={game_type}&range=yearly_"
    url = url + str(year)
    response = r.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    specific_div = soup.find('table') # finding the table with the record of each team
    rows = specific_div.find_all('tr') # get the rows of data

    # Now that we have the data as a list of rows, we can parse the data to construct a data frame
    data = []
    for row in rows:
            cells = row.find_all(['td', 'th'])  # 'td' for regular cells, 'th' for header cells
            row_data = [cell.text.strip() for cell in cells] #extract the contents in each cell
            data.append(row_data)
    columns = data[0]
    df = pd.DataFrame(data[1:], columns=columns)
    df["Year"] = year #helps identify the data points based on year
    return df
read_data("2004")